In [1]:
import pyodbc
import pandas as pd
import numpy as np

### 1. Collect Data

In [3]:
# Get data from local database
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-FK1AJ2V;Trusted_Connection=yes;')
cursor = connection.cursor()

# Get list of transfers
query = "SELECT * FROM la_liga_transfers..transfers_from_api"
cursor.execute(query)
results = cursor.fetchall()

df_transfers = pd.read_sql(query, connection)

# Get stats for players
query = "SELECT * FROM la_liga_transfers..stats_from_season_2015"
cursor.execute(query)
results = cursor.fetchall()

df_stats = pd.read_sql(query, connection)

for season in range(2016, 2023):
    query = f"SELECT * FROM la_liga_transfers..stats_from_season_{season}"
    cursor.execute(query)
    results = cursor.fetchall()

    df_stats = df_stats.append(pd.read_sql(query, connection), ignore_index=True)


cursor.close()
connection.close()



C:\Users\Adrian\AppData\Local\Temp\ipykernel_2704\3604135578.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_transfers = pd.read_sql(query, connection)
C:\Users\Adrian\AppData\Local\Temp\ipykernel_2704\3604135578.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stats = pd.read_sql(query, connection)
C:\Users\Adrian\AppData\Local\Temp\ipykernel_2704\3604135578.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stats = df_stats.append(pd.read_sql(query, connection), ignore_index=True)
C:\Users\Adrian\AppDa

### 2. Data wrangling / Data cleaning

#### 2.1. Df_transfers cleaning

In [4]:
df_transfers.info()
df_transfers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27994 entries, 0 to 27993
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   F1                        27994 non-null  float64
 1   player#id                 27994 non-null  float64
 2   player#name               27993 non-null  object 
 3   transfers#date            27903 non-null  object 
 4   transfers#type            27439 non-null  object 
 5   transfers#teams#in#id     24586 non-null  object 
 6   transfers#teams#in#name   27711 non-null  object 
 7   transfers#teams#in#logo   27703 non-null  object 
 8   transfers#teams#out#id    24265 non-null  object 
 9   transfers#teams#out#name  27747 non-null  object 
 10  transfers#teams#out#logo  27741 non-null  object 
dtypes: float64(2), object(9)
memory usage: 2.3+ MB


,F1,player#id,player#name,transfers#date,transfers#type,transfers#teams#in#id,transfers#teams#in#name,transfers#teams#in#logo,transfers#teams#out#id,transfers#teams#out#name,transfers#teams#out#logo
0,157.0,47203.0,Jaime Romero,2018-07-01 00:00:00,N/A,713.0,Cordoba,https://media-3.api-sports.io/football/teams/7...,716.0,Lugo,https://media-1.api-sports.io/football/teams/7...
1,158.0,47203.0,Jaime Romero,2018-01-17 00:00:00,Loan,716.0,Lugo,https://media-3.api-sports.io/football/teams/7...,713.0,Cordoba,https://media-2.api-sports.io/football/teams/7...
2,159.0,122740.0,J. Torres,2020-08-22 00:00:00,N/A,860.0,Extremadura,https://media-3.api-sports.io/football/teams/8...,716.0,Lugo,https://media-2.api-sports.io/football/teams/7...
3,160.0,122740.0,J. Torres,2019-08-30 00:00:00,Loan,8157.0,FC Andorra,https://media-1.api-sports.io/football/teams/8...,716.0,Lugo,https://media-2.api-sports.io/football/teams/7...
4,161.0,122740.0,J. Torres,2019-08-29 00:00:00,N/A,716.0,Lugo,https://media-3.api-sports.io/football/teams/7...,5268.0,Melilla,https://media-2.api-sports.io/football/teams/5...


In [5]:
# Drop unnecessary columns
df_transfers = df_transfers.drop(df_transfers.columns[[0, 7, 10]], axis=1)

# Get rid of duplicated rows
df_transfers.drop_duplicates(inplace=True)

# Rename column names
df_transfers.columns = [col.replace('#', '_') for col in df_transfers.columns]

# Drop empty data from 'teams in' columns
df_transfers = df_transfers.dropna(subset=['transfers_teams_in_id', 'transfers_teams_in_id'])

##### Convert values in 'transfers_type' column

In [6]:
# Drop nans
df_transfers = df_transfers.dropna(subset='transfers_type')

# Drop loan and swap transfers
mask = df_transfers['transfers_type'].isin(['Loan', 'End of Loan', 'Swap'])
df_transfers = df_transfers.drop(df_transfers[mask].index)

In [7]:
# Function to convert transfers fee expressed as STRING to FLOAT
def convert_to_number(value):
    value = value.upper()
    if value.endswith('M'):
        value = float(value.strip('M'))
    elif value.endswith('K'):
        value = float(value.strip('K')) / 1000
    return value

# Function to convert string data to floats
def convert_transfers_fee(value):
    if value is None:
        return np.nan
    
    if 'Free' in value: # data for free transfers
        value = 0.0
    elif 'N/A' in value: # None values
        value = np.nan
    else: # convert string data to float ()
        if '€' in value:
            value = value.strip('€').strip()
            value = convert_to_number(value)
        elif '$' in value:
            value = value.strip('$').strip()
            value = round(convert_to_number(value) * 0.92, 2) # convert to EU (exchange rate on 25.06.23)
        else:
            value = convert_to_number(value)
    
    return value
         
                  
df_transfers['transfers_type'] = df_transfers['transfers_type'].apply(convert_transfers_fee) 

# Column rename
df_transfers = df_transfers.rename(columns={'transfers_type': 'transfer_fee'})

In [8]:
# Change Series types
df_transfers['player_id'] = df_transfers['player_id'].astype('int')
df_transfers['transfers_teams_in_id'] = pd.to_numeric(df_transfers['transfers_teams_in_id']).astype('int')
df_transfers['transfers_date'] = pd.to_datetime(df_transfers['transfers_date'], errors='coerce') 

# Reset indexes
df_transfers = df_transfers.reset_index(drop=True)


##### Create new column for season of transfer

In [10]:
# --- Season 2018 is 2018/2019
# Summer transfer window is 18-07-01 - 18-09-01
# Winter transfer window is 19-01-02 - 19-02-01
# 
# So for every year:
# January, February, March - season of transfer is ({year} - 1)
# April to December        - season of transfer is {year}
# -----------

def season_of_transfer(row):
    if row['transfers_date'] is None:
        return np.nan
    else:
        if row['transfers_date'].month in range(1, 4):
            return row['transfers_date'].year - 1
        else:
            return row['transfers_date'].year
            

df_transfers['season_of_transfer'] = df_transfers.apply(season_of_transfer, axis=1)


In [11]:
df_transfers.info()
df_transfers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15991 entries, 0 to 15990
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   player_id                 15991 non-null  int32         
 1   player_name               15991 non-null  object        
 2   transfers_date            15956 non-null  datetime64[ns]
 3   transfer_fee              6105 non-null   float64       
 4   transfers_teams_in_id     15991 non-null  int32         
 5   transfers_teams_in_name   15991 non-null  object        
 6   transfers_teams_out_id    13784 non-null  object        
 7   transfers_teams_out_name  15851 non-null  object        
 8   season_of_transfer        15956 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(2), object(4)
memory usage: 999.6+ KB


,player_id,player_name,transfers_date,transfer_fee,transfers_teams_in_id,transfers_teams_in_name,transfers_teams_out_id,transfers_teams_out_name,season_of_transfer
0,47203,Jaime Romero,2018-07-01,NaN,713,Cordoba,716.0,Lugo,2018.0
1,122740,J. Torres,2020-08-22,NaN,860,Extremadura,716.0,Lugo,2020.0
2,122740,J. Torres,2019-08-29,NaN,716,Lugo,5268.0,Melilla,2019.0


#### 2.2 Df_stats cleaning

In [12]:
df_stats.info()
df_stats.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5793 entries, 0 to 5792
Data columns (total 60 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   F1                                5784 non-null   float64       
 1   player#id                         5784 non-null   float64       
 2   player#name                       5790 non-null   object        
 3   player#firstname                  5782 non-null   object        
 4   player#lastname                   5786 non-null   object        
 5   player#age                        5781 non-null   float64       
 6   player#birth#date                 5785 non-null   datetime64[ns]
 7   player#birth#place                5626 non-null   object        
 8   player#birth#country              5784 non-null   object        
 9   player#nationality                5784 non-null   object        
 10  player#height                     5535 non-null 

,F1,player#id,player#name,player#firstname,player#lastname,player#age,player#birth#date,player#birth#place,player#birth#country,player#nationality,...,statistics#fouls#drawn,statistics#fouls#committed,statistics#cards#yellow,statistics#cards#yellowred,statistics#cards#red,statistics#penalty#won,statistics#penalty#commited,statistics#penalty#scored,statistics#penalty#missed,statistics#penalty#saved
0,0.0,367.0,Iker Casillas Fernández,Iker,Casillas Fernández,38.0,1981-05-20,Móstoles,Spain,Spain,...,None,None,0.0,0.0,0.0,None,None,None,None,None
1,1.0,1630.0,M. Musacchio,Mateo Pablo,Musacchio,31.0,1990-08-26,Rosario,Argentina,Argentina,...,9.0,13.0,1.0,0.0,0.0,None,None,0.0,0.0,None
2,2.0,46784.0,Dani Giménez,Daniel,Giménez Hernández,38.0,1983-07-30,Vigo,Spain,Spain,...,0.0,0.0,0.0,0.0,0.0,None,None,0.0,0.0,1.0


In [13]:
# Drop unnecessary columns
col_to_drop = [0, 26, 29, 32] + list(range(6, 9)) + list(range(11, 15)) + list(range(16, 22))
df_stats = df_stats.drop(df_stats.columns[col_to_drop], axis=1)

# Rename column names
df_stats.columns = [col.replace('#', '_') for col in df_stats.columns]

# Get rid of duplicated rows
df_stats.drop_duplicates(inplace=True)

# Drop rows with None values in 'team_name'
mask = df_stats['statistics_team_name'].isna()
df_stats = df_stats.drop(df_stats[mask].index)

In [14]:
# Change  columns types
df_stats[df_stats.columns[[0, 4, 8]]] = df_stats[df_stats.columns[[0, 4, 8]]].astype('int')
df_stats[df_stats.columns[list(range(9, 12)) + list(range(13, 43))]] = df_stats[df_stats.columns[list(range(9, 12)) + list(range(13, 43))]].astype('float')


In [15]:
# Convert 'player_height' columns as float

def convert_player_height(value):
    if value is None:
        return np.nan
    else:
        value = value.strip('cm').strip()
    return value

df_stats['player_height'] = df_stats['player_height'].apply(convert_player_height).astype('float')


In [16]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5723 entries, 0 to 5792
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   player_id                         5723 non-null   int32  
 1   player_name                       5723 non-null   object 
 2   player_firstname                  5723 non-null   object 
 3   player_lastname                   5723 non-null   object 
 4   player_age                        5723 non-null   int32  
 5   player_nationality                5723 non-null   object 
 6   player_height                     5477 non-null   float64
 7   statistics_team_name              5723 non-null   object 
 8   statistics_league_season          5723 non-null   int32  
 9   statistics_games_appearences      5692 non-null   float64
 10  statistics_games_lineups          5692 non-null   float64
 11  statistics_games_minutes          5692 non-null   float64
 12  statis

#### 2.3 Merge Data-sets
- {merged_df} - List of players transferred TO La liga clubs in the seasons from 2015/16 to 2022/23 and  their statistics in the first season after transfer

In [27]:
merged_df = df_transfers.merge(df_stats, how='inner', 
                               left_on=['player_id', 'player_name', 'transfers_teams_in_name', 'season_of_transfer'],
                               right_on=['player_id', 'player_name', 'statistics_team_name', 'statistics_league_season'])

In [29]:
merged_df.describe().round(2)

,player_id,transfer_fee,transfers_teams_in_id,season_of_transfer,player_age,player_height,statistics_league_season,statistics_games_appearences,statistics_games_lineups,statistics_games_minutes,...,statistics_fouls_drawn,statistics_fouls_committed,statistics_cards_yellow,statistics_cards_yellowred,statistics_cards_red,statistics_penalty_won,statistics_penalty_commited,statistics_penalty_scored,statistics_penalty_missed,statistics_penalty_saved
count,1290.00,721.00,1290.00,1290.00,1290.00,1273.00,1290.00,1288.00,1288.00,1288.00,...,1019.00,1030.00,1288.00,1288.00,1288.00,85.00,91.00,1119.00,1119.00,95.00
mean,40875.49,6.71,581.47,2018.46,30.04,181.49,2018.46,15.30,11.02,992.70,...,16.29,18.30,2.63,0.06,0.06,1.31,1.30,0.11,0.03,0.58
std,48004.56,14.15,83.62,2.29,4.21,6.38,2.29,12.18,10.72,933.62,...,16.20,16.14,3.10,0.25,0.25,0.71,0.53,0.51,0.18,0.79
min,15.00,0.00,529.00,2015.00,19.00,163.00,2015.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,2044.25,0.00,534.00,2016.00,27.00,177.00,2016.00,3.00,1.00,90.00,...,4.00,5.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
50%,46786.00,1.50,541.00,2018.00,30.00,182.00,2018.00,15.00,8.00,788.50,...,11.00,15.00,2.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
75%,47414.75,7.00,548.00,2020.00,33.00,186.00,2020.00,27.00,19.00,1710.00,...,23.00,27.00,4.00,0.00,0.00,1.00,2.00,0.00,0.00,1.00
max,336594.00,126.00,798.00,2022.00,42.00,200.00,2022.00,38.00,38.00,3420.00,...,111.00,88.00,17.00,2.00,3.00,5.00,3.00,5.00,2.00,4.00
